## PROJECT. Исследование поведения пользователей

### ЗАДАНИЕ
необходимо проверить:
* есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
* различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

In [213]:
import pandas as pd

In [214]:
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')


In [215]:
event_2018 = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
registered = events_df[event_2018]['user_id'].to_list()  # список пользователей, зарег. в 2018
events = events_df[events_df.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')

D:\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [216]:
purchase = purchase_df[purchase_df.user_id.isin(registered)]
purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')
purchase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


In [217]:
#purchase['event_type'] = 'purchase'

In [218]:
events = events.rename(columns={"id": "event_id"})
purchase = purchase.rename(columns={"id": "purchase_id"})

In [219]:
total_events_df = pd.concat([events,purchase],sort=False)
total_events_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
51410,80313.0,registration,NaN,2018-01-01 14:08:40,NaN,27837,NaN,NaT,NaN
51411,80314.0,registration,NaN,2018-01-01 14:42:58,NaN,27838,NaN,NaT,NaN
51412,80315.0,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaN,NaT,NaN
51413,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaT,NaN
51414,80317.0,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaN,NaT,NaN


In [220]:
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

In [221]:
# посмотрим уникальные значения в датафрейме events, в event_type
events['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

In [222]:
# посмотрим какие уровни сложности имеются. Уникальные значения
events['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

Посмотрим сколько всего пользователей выбирают уровень сложности:
* easy
* medium
* hard

In [223]:
events[events['selected_level'] == 'easy']['user_id'].nunique()

2448

### Проверим гипотезы

Найдем пользователей, которые оплатили и выбрали определенный уровень сложности

определим процент пользователей, которые выбрали определенный уровень сложности и дошли до оплаты, и запишем его в переменную percent_level_choice_users.

In [224]:
level_choice_easy_user_count = events[events['selected_level'] == 'easy']['user_id'].nunique()
level_choice_easy_user_count

2448

In [236]:
level_choice_easy_user_count_1 = total_events_df[
    total_events_df["selected_level"] == "easy"
]["user_id"].unique()
print(len(level_choice_easy_user_count_1))

2448


In [237]:
level_choice_easy_user_count = purchase_df[purchase_df["user_id"].isin(level_choice_easy_user_count_1)]
level_choice_easy_user_count['user_id'].nunique() #  посчитаем, сколько пользователей в этом датафрейме.

In [248]:
paying_users_count = purchase['user_id'].nunique()

percent_of_paying_users = len(level_choice_easy_user_count) / paying_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень easy): {:.2%}".format(
        percent_of_paying_users
    )
)


Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень easy): 11.81%


In [249]:
level_choice_medium_user_count = events[events['selected_level'] == 'medium']['user_id'].nunique()
level_choice_medium_user_count

4645

In [250]:
percent_of_paying_users =  paying_users_count / level_choice_medium_user_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень medium): {:.2%}".format(
        percent_of_paying_users
    )
)


Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень medium): 34.45%


In [251]:
level_choice_hard_user_count = events[events['selected_level'] == 'hard']['user_id'].nunique()

In [252]:
percent_of_paying_users = level_choice_hard_user_count / paying_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень hard): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень hard): 78.06%
